In [ ]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
import librosa.display
import os
import pandas as pd

In [ ]:
# make dataframe to store each filename, audio array (data), and sample rate (sr) of GTZAN dataset

directory = 'genres'

df = pd.DataFrame(columns =['filename','data','sr'])

for folders in os.listdir(directory):
    # folders in 'genres'
    if folders != '.DS_Store':
        for filename in os.listdir(directory+"/"+folders):
            f = os.path.join(directory, folders, filename)
            data, sr = librosa.load(f)
            insert_data = {'filename':str(f), 'data': data, 'sr': sr}
            df = df.append(insert_data, ignore_index=True)

In [ ]:
# make directories to store mel spectrograms

genres = ['blues', 'classical', 'country', 'disco', 'pop', 'hiphop', 'metal', 'reggae', 'rock', 'jazz']

main_directory = 'mel_spectrograms_3sec/'

for g in genres:
    name = main_directory + g
    os.makedirs(name)

In [ ]:
#make mel spectrogram for each audio file
#change split_size to determine length of audio file used in spectrogram (661794 is array length of 30 sec audio)
#chop off the last 4 elements of array for even division into 10 segements


for fn, data, sr in zip(df['filename'], df['data'], df['sr']):
    #split_size = int(len(data)/3) #split 30 second audio data into 3 10-second segements
    split_size = int(661790/10) #split 30 second audio data into 10 3-second segments
    
    path, num, ext = fn.split('.')
    top_dir, mid_dir, low_dir = path.split('/')
    new_path = main_directory + "/" + mid_dir + "/" + low_dir
    
    for i in range(10):
        fig, ax = plt.subplots()
        ax.set_axis_off()
        
        #make the mel spectrogram
        split_data = data[i*split_size:(i+1)*split_size]
        S = librosa.feature.melspectrogram(y=split_data, sr=sr, n_mels=128, fmax=8000)
        S_dB = librosa.power_to_db(S, ref=np.max)
        img = librosa.display.specshow(S_dB, x_axis='time', y_axis='mel', sr=sr, fmax=8000);
        
        #save it in new directory mel_spectrograms/*genre name*/*genre name*.number_split.png
        filename = new_path + '.' + num + '_' + str(i) + '.png'
        
        fig.savefig(filename, bbox_inches='tight', transparent=True, pad_inches=0.0)
        fig.clf()
        plt.close(fig)